<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1:
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `http://....`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Objetivos principales del laboratorio](#Objetivos-principales-del-laboratorio)

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Obtener datos desde Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Optimizar a través de librerías funciones.

#Importamos librerias utiles 😸

In [1]:
%%capture
# !pip install praw
# !pip install line_profiler
# !pip install numba

import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://styles.redditmedia.com/t5_2rer8/styles/bannerBackgroundImage_xzrb6my122181.jpg">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada liena de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [ ] Estudiar la memoria ocupada por el código a través de un perfilador.
- [ ] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

In [2]:
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url


In [3]:
votes, post, url = praw_reddit()

In [4]:
#Al utilizar %lprun y %mprun se espera que logren visualizar que la consulta al servidor sea lo que mas tiempo gasta en el codigo.
#A la vez se espera que entregan los resultados de los perfiladores que demuestran esto, deben entregar tanto %lprun, como %mprun
#esto puede cambiar harto dependiendo del pc que ocuparon.

**Respuesta:**

`Escriba su respuesta aquí`

## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder las preguntas.
- [ ] Mejorar el código con cache.

**Respuestas Teóricas:**

`Escriba su respuesta aquí`

In [5]:
#Solo ocupar @lru_cache(maxsize=1000) y comentar que sucede 
# si se logra aumentar la velocidad de la función. 
#Algo que deberian notar es que no entrega valores actualizados el uso de cache.

## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [4]:
from numba import jit

def intervals(data):
    mu, std_err = [], []
    for u, d in data:
        a = 1 + u
        b = 1 + d
        mu.append(a / (a + b))
        std_err.append(1.65 * ((a*b) / ((a+b)**2*(a+b+1)))**(1/2))
    return (mu, std_err)

def intervals_numpy(u, d): #Sin uso de "for" aquí.
    a = 1. + u
    b = 1. + d
    mu = a / (a+b)
    std_err = 1.65 * np.sqrt((a*b) / ((a+b)**2*(a+b+1)))
    return (mu, std_err)


## 1.4 Comparación de Rendimiento [2 Puntos]

Creadas las funciones de la sección 1.3, de forma iterativa compare los diferentes batches de datos que contiene el diccionario `votes`. Para esto genere un gráfico utilizando plotly, donde se pueda observar las diferencias de tiempo que toma ejecutar las diferentes cantidades de datos. ¿es posible observar una diferencia? ¿a qué se deberá esto?.

Aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. Grafique estos tiempos y observe comente los desempeños.

In [5]:
###### Código Aquí ######

@jit(nopython=True)
def intervals_JIT(data):
    mu, std_err = [], []
    for u, d in data:
        a = 1. + u
        b = 1. + d
        mu.append(a / (a + b))
        std_err.append(1.65 * ((a*b) / ((a + b)**2*(a+b+1.)))**(1/2))
    return (mu, std_err)


@jit(nopython=True)
def intervals_numpy_JIT(u, d):
    a = 1. + u
    b = 1. + d
    mu = a / (a+b)
    std_err = 1.65 * np.sqrt((a*b) / ((a+b)**2*(a+b+1.)))
    return (mu, std_err)

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

In [8]:
import time
# Se busca que basicamente hagan el plot
tiempo, tipo, iteration = [],[],[]
for it in range(len(votes)):
    t0 = time.process_time()
    intervals(votes[it])
    t1 = time.process_time()
    tiempo.append(t1-t0)
    tipo.append('Python')
    iteration.append(len(votes[it]))
    up, down = votes[it][:, 0], votes[it][:, 1]
    t0 = time.process_time()
    intervals_numpy(up, down)
    t1 = time.process_time()
    tiempo.append(t1-t0)
    tipo.append('Numpy')
    iteration.append(len(votes[it]))
    t0 = time.process_time()
    intervals_JIT(votes[it])
    t1 = time.process_time()
    tiempo.append(t1-t0)
    tipo.append('Python-JIT')
    iteration.append(len(votes[it]))
    t0 = time.process_time()
    intervals_numpy_JIT(up, down)
    t1 = time.process_time()
    tiempo.append(t1-t0)
    tipo.append('Numpy-JIT')
    iteration.append(len(votes[it]))
data_plot = {'it':iteration, 'Tiempo':tiempo, 'tipo':tipo}
df = pd.DataFrame(data=data_plot)
fig = px.line(df, x="it", y="Tiempo", color='tipo')

fig.show()

<ipython-input-4-8a9df3807b03>:9: RuntimeWarning:

overflow encountered in long_scalars

<ipython-input-4-8a9df3807b03>:9: RuntimeWarning:

invalid value encountered in double_scalars



In [7]:
fig.write_html('./medicion_tiempos.html')

## 1.5 Plot de Resultados [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [10]:
votos = votes[len(votes) - 1]
posteo = post[len(post) - 1]
print("lower bounds aproximados:")
posterior_mean, std_err = intervals_numpy(votos[:, 0], votos[:, 1])
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {
    'Votos (+)': votos[order[:20], 0],
    'Votos (-)': votos[order[:20], 1],
    'Post': np.array(posteo)[order[:20]],
    'url': np.array(url[len(url) - 1])[order[:20]]
}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post

lower bounds aproximados:

Top 20 post ordenador por el limite inferior:



In [11]:
r_order = order[::-1][-20:]
data_dic = {
    'mean': posterior_mean[r_order],
    'std_err': std_err[r_order],
    'post': ordered_post[::-1]
}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post", error_x="std_err")
fig.show()

In [12]:
fig.write_html('./posts.html')

**Respuesta:**

Que comenten solamente lo que observan.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>